# InDiE DialogFlow publisher POC

InDiE (Intent Discovery Engine) is a module that builds a tree of intents using supplied SQL queries and corresponding utterances. It also provides fulfillment logic to convert intent and slot values into a SQL query.

## DialogFlow publisher
The DialogFlow (DF) publisher module takes the list of intents from a simplified InDiE json file. It uses DialogFlow APIs to create corresponding intents in a DialogFlow agent. It also configures DF agent to ask the user for any missing slots. Custom entities are also published so DF can detect particular types of entities from user utterance.


In [1]:
%load_ext autoreload
%autoreload 1

!mkdir indie_df_publisher

mkdir: indie_df_publisher: File exists


In [2]:
%%writefile indie_df_publisher/dialogflow_agent.py
import dialogflow_v2 as dialogflow

class DialogFlowAgent:
    def __init__(self, namespace, project):
        self.namespace = namespace
        self.project = project
        self.agents_client = dialogflow.AgentsClient()
        self.intents_client = dialogflow.IntentsClient()

        self.agent = self.load_agent()

    def load_agent(self):
        parent = self.agents_client.project_path(self.project)
        agent = self.agents_client.get_agent(parent)
        return agent

    def set_default_agent(self):
        parent = self.agents_client.project_path(self.project)

        a2 = dialogflow.types.Agent()
        a2.parent = parent
        a2.display_name = f"{self.namespace} agent"
        a2.default_language_code = "en"
        a2.time_zone = "America/Denver"
        a2.classification_threshold = 0.3
        self.agent = agents_client.set_agent(a2)
        return self.agent
    
    def list_intents(self):
        parent = self.intents_client.project_agent_path(self.project)
        return {x.display_name:x for x in self.intents_client.list_intents(parent)}

    def delete_indie_intents(self):
        parent = self.intents_client.project_agent_path(self.project)

        intents = self.list_intents()
        intents_to_delete = []
        for display_name in intents:
            if display_name.startswith(self.namespace):
                intents_to_delete.append(intents[display_name])
        if len(intents_to_delete) > 0:
            self.intents_client.batch_delete_intents(parent, intents_to_delete)
        
    def build_intent(self, display_name_suffix, training_phrases=None,
                      message_texts=None, parameters=None, is_fallback=False,
                      parent_followup_intent_name=None):
        parent = self.intents_client.project_agent_path(self.project)

        messages = []
        if message_texts:
            text = dialogflow.types.Intent.Message.Text(text=message_texts)
            message = dialogflow.types.Intent.Message(text=text)
            messages.append(message)

        intent = dialogflow.types.Intent(
            display_name=f"{self.namespace}_{display_name_suffix}",
            training_phrases=training_phrases,
            messages=messages,
            parameters=parameters,
            is_fallback=is_fallback,
            parent_followup_intent_name=parent_followup_intent_name)
        
        intent.webhook_state = dialogflow.types.Intent.WebhookState.WEBHOOK_STATE_ENABLED

        return intent

    def create_intents(self, intents):
        parent = self.intents_client.project_agent_path(self.project)

        intents_batch =  dialogflow.types.IntentBatch()
        intents_batch.intents.extend(intents)
        response = self.intents_client.batch_update_intents(parent, intent_batch_inline=intents_batch)

        def callback(operation_future):
            # Handle result.
            result = operation_future.result()
            print('Intents created:')
            print(result)

        response.add_done_callback(callback)
        
    def train(self):
        parent = self.agents_client.project_path(self.project)

        response = self.agents_client.train_agent(parent)
        def callback(operation_future):
            # Handle result.
            result = operation_future.result()
            print('Agent training:')
            print(result)

        response.add_done_callback(callback)


Overwriting indie_df_publisher/dialogflow_agent.py


In [3]:
%%writefile indie_df_publisher/entity_types_publisher.py
import dialogflow_v2 as dialogflow

class EntityTypesPublisher:
    def __init__(self, namespace, project):
        self.client = dialogflow.EntityTypesClient()
        self.project = project
        self.namespace = namespace
        self.table_column_to_custom_entity = {}

    def create_entity_types(self, indie_custom_entities, indie):
        parent = self.client.project_agent_path(self.project)

        entity_type_batch = dialogflow.types.EntityTypeBatch()
        for de in indie_custom_entities:
            et = self.indie_custom_entity_to_dialogflow_entity_type(de)
            if et:
                entity_type_batch.entity_types.append(et)

        response = self.client.batch_update_entity_types(parent, entity_type_batch_inline=entity_type_batch)

        def callback(operation_future):
            # Handle result.
            result = operation_future.result()
            print('Entity types created')
            print(result)

        response.add_done_callback(callback)
        
    def indie_custom_entity_to_dialogflow_entity_type(self, indie_custom_entity):
        entity_type = dialogflow.types.EntityType()
        entity_type.display_name = f"{self.namespace}-{indie_custom_entity['index']}"
        self.table_column_to_custom_entity[f"{indie_custom_entity['table']}.{indie_custom_entity['column']}"] = f"@{entity_type.display_name}"
        entity_type.kind = 1
        
        for value, synonyms in indie_custom_entity['dictionary'].items():
            entity = dialogflow.types.EntityType.Entity()
            entity.value = value
            if len(synonyms) > 0:
                entity.synonyms.extend(synonyms)
            entity_type.entities.append(entity)
        
        return entity_type

    def list_entity_types(self):
        parent = self.client.project_agent_path(self.project)
        return {x.display_name:x for x in self.client.list_entity_types(parent)}
    
    def delete_indie_entity_types(self):
        parent = self.client.project_agent_path(self.project)

        entity_types = self.list_entity_types()
        entity_types_to_delete = []
        for display_name in entity_types:
            if display_name.startswith(self.namespace):
                entity_types_to_delete.append(entity_types[display_name].name)
        if len(entity_types_to_delete) > 0:
            self.client.batch_delete_entity_types(parent, entity_types_to_delete)
        

Overwriting indie_df_publisher/entity_types_publisher.py


In [4]:
%%writefile indie_df_publisher/dialogflow_publisher.py

from indie_df_publisher.dialogflow_agent import DialogFlowAgent
from indie_df_publisher.entity_types_publisher import EntityTypesPublisher

import dialogflow_v2 as dialogflow

class DialogFlowPublisher:
    def __init__(self, namespace, project):
        self.namespace = namespace
        self.dialogflow_agent = DialogFlowAgent(namespace, project)
        self.entity_types_publisher = EntityTypesPublisher(namespace, project)
    
    def indie_training_phrases_to_dialogflow_training_phrases(self, indie_training_phrases):
        return [self.indie_training_phrase_to_dialogflow_training_phrase(tp) for tp in indie_training_phrases]
        
    def indie_training_phrase_to_dialogflow_training_phrase(self, indie_training_phrase):
        tp = dialogflow.types.Intent.TrainingPhrase()
        for part in indie_training_phrase['parts']:
            tpp = dialogflow.types.Intent.TrainingPhrase.Part()
            tpp.text = part['text']
            
            if 'alias' in part:
                tpp.alias = part['alias']

            if 'entity_type' in part:
                tpp.entity_type = part['entity_type']

            tp.parts.append(tpp)

        return tp    

    def indie_intent_to_dialogflow_intent(self, indie_intent, indie):
        display_name_suffix = indie_intent['id']

        training_phrases = self.indie_training_phrases_to_dialogflow_training_phrases(indie_intent['utterances'])
        followup_training_phrases = self.indie_training_phrases_to_dialogflow_training_phrases(indie_intent['followup_utterances'])

        parameters = []
        for indie_parameter in indie_intent['parameters']:
            parameter = dialogflow.types.Intent.Parameter(
                display_name=indie_parameter['id'],
                value=f"${indie_parameter['id']}",
                is_list=True,
                mandatory=indie_parameter['mandatory'],
                entity_type_display_name=indie_parameter['entity_type'],
                prompts=[f"What {indie_parameter['friendly_name']}?"])
            parameters.append(parameter)
            
        message = indie_intent['response_template']
        
        main_intent = self.dialogflow_agent.build_intent(display_name_suffix=display_name_suffix,
                                                  training_phrases=training_phrases,
                                                  message_texts=[message],
                                                  parameters=parameters,
                                                  is_fallback=False)

        followup_intent = self.dialogflow_agent.build_intent(display_name_suffix=display_name_suffix + '_followup',
                                                  training_phrases=followup_training_phrases,
                                                  message_texts=[message],
                                                  parameters=parameters,
                                                  is_fallback=False,
                                                  parent_followup_intent_name=main_intent.display_name)

        return main_intent, followup_intent
        

    def publish(self, indie):
        self.dialogflow_agent.delete_indie_intents()
        
        self.entity_types_publisher.delete_indie_entity_types()
        self.entity_types_publisher.create_entity_types(indie['custom_entities'].values(), indie)

        df_intents = []
        df_followup_intents = []
        indie_intents = indie['intents']
        
        for indie_intent in indie_intents:
            main_intent, followup_intent = self.indie_intent_to_dialogflow_intent(indie_intent, indie)
            df_intents.append(main_intent)
            if followup_intent:
                df_followup_intents.append(followup_intent)

        self.dialogflow_agent.create_intents(df_intents)
        intents = self.dialogflow_agent.list_intents()
        
        for df_followup_intent in df_followup_intents:
            df_followup_intent.parent_followup_intent_name = intents[df_followup_intent.parent_followup_intent_name].name

        self.dialogflow_agent.create_intents(df_followup_intents)
        
        self.dialogflow_agent.train()

Overwriting indie_df_publisher/dialogflow_publisher.py


In [6]:
# %%writefile indie_df_publisher/publish.py

%aimport indie_df_publisher.dialogflow_agent
%aimport indie_df_publisher.entity_types_publisher
%aimport indie_df_publisher.dialogflow_publisher

from indie_df_publisher.dialogflow_publisher import DialogFlowPublisher

import dialogflow_v2 as dialogflow # https://dialogflow-python-client-v2.readthedocs.io/en/latest/gapic/v2beta1/api.html
import json

class args: pass

# Configuration to access and modify a DialogFlow agent
%env GOOGLE_APPLICATION_CREDENTIALS=/Users/amolk/work/gotit/tutoruniverse/dialogflow_publisher/k3-oekhkw-65c192c69f18.json
args.dialogflow_project = "k3-oekhkw"
# args.simple_indie_format_file_path = "../data/InDiE_0.3_CCAI_20200608155202.simple.json"
args.simple_indie_format_file_path = "/Users/amolk/work/gotit/tutoruniverse/dialogflow_publisher/data/InDiE_0.3_Echo_20200612184151.simple.json"

publisher = DialogFlowPublisher(namespace='gotit', project=args.dialogflow_project)

with open(args.simple_indie_format_file_path) as json_file:
    simple_indie = json.load(json_file)

publisher.publish(simple_indie)


env: GOOGLE_APPLICATION_CREDENTIALS=/Users/amolk/work/gotit/tutoruniverse/dialogflow_publisher/k3-oekhkw-65c192c69f18.json
Entity types created
entity_types {
  name: "projects/k3-oekhkw/agent/entityTypes/775c2996-9abb-4b2a-90e2-0e9eeb239724"
  display_name: "gotit-0"
  kind: KIND_MAP
  entities {
    value: "online"
    synonyms: "online"
  }
  entities {
    value: "store"
    synonyms: "store"
  }
}
entity_types {
  name: "projects/k3-oekhkw/agent/entityTypes/8cc4cab4-a1da-48ee-b6d0-32fa18c0e334"
  display_name: "gotit-1"
  kind: KIND_MAP
  entities {
    value: "10GB plan"
    synonyms: "10GB plan"
  }
  entities {
    value: "earphones"
    synonyms: "earphones"
  }
  entities {
    value: "family plan"
    synonyms: "family plan"
  }
  entities {
    value: "ipad"
    synonyms: "ipad"
  }
  entities {
    value: "ipad pro"
    synonyms: "ipad pro"
  }
  entities {
    value: "iphone 11"
    synonyms: "iphone 11"
  }
  entities {
    value: "iphone 11 pro"
    synonyms: "iphone 11